In [1]:
import os
import json
import numpy as np

In [8]:
def get_best_mmds(run_dir):
    # First, get the best run based on last loss
    best_loss, best_metrics = float("inf"), None
    for run_num in os.listdir(run_dir):
        if run_num == "_sources":
            continue
        metrics_path = os.path.join(run_dir, run_num, "metrics.json")
        with open(metrics_path, "r") as f:
            metrics = json.load(f)
        try:
            last_loss = metrics["train_epoch_loss"]["values"][-1]
            
            # Try and get a metric out
            _ = metrics["orbit_mmd"]["values"]
        except KeyError:
            print("Warning: Did not find finished run in %s" % os.path.join(run_dir, run_num))
            last_loss = float("inf")
        if last_loss < best_loss:
            best_loss, best_metrics = last_loss, metrics
    
    # Now return the MMDs and baselines
    return (
        [
            best_metrics["degree_mmd"]["values"][0],
            best_metrics["cluster_coef_mmd"]["values"][0],
#             best_metrics["spectra_mmd"]["values"][0],
            best_metrics["orbit_mmd"]["values"][0]
        ],
        [
            best_metrics["degree_mmd_baseline"]["values"][0],
            best_metrics["cluster_coef_mmd_baseline"]["values"][0],
#             best_metrics["spectra_mmd_baseline"]["values"][0],
            best_metrics["orbit_mmd_baseline"]["values"][0]
        ]
    )

In [9]:
# Import MMD and baseline values from training runs

base_path = "/gstore/home/tsenga5/discrete_graph_diffusion/models/trained_models/"
my_mmds_and_baselines = {
    "Community (small)": {
        "Edge-flip": get_best_mmds(os.path.join(base_path, "benchmark-nocache_community-small_edge-flip")),
        "Edge-one": get_best_mmds(os.path.join(base_path, "benchmark-nocache_community-small_edge-addition")),
        "Edge-zero": get_best_mmds(os.path.join(base_path, "benchmark-nocache_community-small_edge-deletion"))
    },
    "Stochastic block models": {
        "Edge-flip": get_best_mmds(os.path.join(base_path, "benchmark-nocache_sbm_edge-flip")),
        "Edge-one": get_best_mmds(os.path.join(base_path, "benchmark-nocache_sbm_edge-addition")),
        "Edge-zero": get_best_mmds(os.path.join(base_path, "benchmark-nocache_sbm_edge-deletion"))
    }
}

my_mmds = {d_key : {k_key : vals[0] for k_key, vals in d_dict.items()} for d_key, d_dict in my_mmds_and_baselines.items()}
my_baselines = {d_key : {k_key : vals[1] for k_key, vals in d_dict.items()} for d_key, d_dict in my_mmds_and_baselines.items()}

In [13]:
benchmark_baselines = {  # From SPECTRE paper
    "Community (small)": [0.02, 0.07, 0.01],
    "Stochastic block models": [0.0008, 0.0332, 0.0255]
}

In [15]:
# Print out results

def print_vals(key, vals):
    vals =  np.sqrt(vals)
    print("%s & %.2f & %.2f & %.2f" % (key, vals[0], vals[1], vals[3]))

for d_key in my_mmds.keys():
    print(d_key)    
    for my_key, my_vals in my_mmds[d_key].items():
        v = np.array(my_vals)
        b = np.array(my_baselines[d_key][my_key])
        b = np.array(benchmark_baselines[d_key])
        s = my_key + " & "
        s += " & ".join(["%.4f" % x for x in v]) + " & "
#         s += " & ".join(["%.4f" % x for x in b]) + " & "
        s += " & ".join(["%.2f" % x for x in (v / b)]) + " \\\\"
        print(s)
    print("=========================")

Community (small)
Edge-flip & 0.0072 & 0.0319 & 0.0033 & 0.36 & 0.46 & 0.33 \\
Edge-one & 0.1913 & 0.1332 & 0.2977 & 9.56 & 1.90 & 29.77 \\
Edge-zero & 0.0171 & 0.0274 & 0.0046 & 0.85 & 0.39 & 0.46 \\
Stochastic block models
Edge-flip & 0.0409 & 0.0359 & 0.0147 & 51.17 & 1.08 & 0.58 \\
Edge-one & 0.0313 & 0.0328 & 0.0160 & 39.16 & 0.99 & 0.63 \\
Edge-zero & 0.0013 & 0.0337 & 0.0168 & 1.59 & 1.02 & 0.66 \\
